In [1]:
%load_ext autoreload

%autoreload 2

import dns_weekend

In [2]:
import pytest

known = 'google.com'
buf = b''.join(dns_weekend._encode_name(known))

name, offset = dns_weekend._decode_name(buf)
print(f'{known=} {buf=} {name=} {offset=}')

assert name == known
assert buf[offset:] == b''

# make sure compression works
name, offset = dns_weekend._decode_name(b'\xc0\x02\x07example\x03com\x00')
assert name == 'example.com'
assert offset == 2

# make sure infinite references are rejected
for test in (b'\xc0\x00', b'\x07example\x03com\xc0\x08'):
    with pytest.raises(ValueError):
        dns_weekend._decode_name(test)

known='google.com' buf=b'\x06google\x03com\x00' name='google.com' offset=12


In [3]:
query = dns_weekend.make_question('example.com')
print(query)
print(dns_weekend.decode_response(query))

b'M\x0f\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x07example\x03com\x00\x00\x01\x00\x01'
Header(id=19727, flags=256, questions=[Question(name='example.com', type_=<DnsType.A: 1>, class_=<DnsClass.IN: 1>)], answers=[], authorities=[], additionals=[])


In [4]:
import socket

req1, req2 = (
    dns_weekend.make_question('www.facebook.com', qtype=t)
    for t in 'A AAAA'.split()
)

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
    sock.connect(("8.8.8.8", 53))

    sock.send(req1)
    sock.send(req2)

    res1 = sock.recv(1024)
    res2 = sock.recv(1024)

for ans in dns_weekend.decode_response(res1).answers:
    print(ans)

for ans in dns_weekend.decode_response(res2).answers:
    print(ans)

RecordInetCNAME(name='www.facebook.com', ttl=1250, target='star-mini.c10r.facebook.com')
RecordInetAAAA(name='star-mini.c10r.facebook.com', ttl=60, address='2a03:2880:f158:82:face:b00c:0:25de')
RecordInetCNAME(name='www.facebook.com', ttl=1250, target='star-mini.c10r.facebook.com')
RecordInetA(name='star-mini.c10r.facebook.com', ttl=60, address='157.240.221.35')
